# Dynex G70

In [1]:
import dynex
import networkx as nx
import matplotlib
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
import dimod
from pyqubo import Spin, Array
from time import sleep
import pickle

In [2]:
filename = 'G70.dat' 

## Load Graph from File

In [3]:
# build graph from file:
G = nx.Graph();
f = open(filename, "r").readlines()
for ele in f[1:]:
    i, j, v = ele.split()
    G.add_edges_from([(int(i),int(j))])

## Construct QUBO

In [4]:
n = 10001 
X = Array.create('X', n, vartype='SPIN')
H0 = 0
for (u,v) in G.edges:
    H0 -= (1-X[u]*X[v])/2.0

In [5]:
model = H0.compile()

In [6]:
bqm = model.to_bqm()
factor = bqm.normalize()
print('bqm was normalized with factor',factor)

bqm was normalized with factor 0.1111111111111111


## Sample Sweep

In [7]:
best_cut = 0
best_sol = [];

In [ ]:
all_sol = [];

for i in range(1,10000000):
    # Sample on Dynex
    print('Sweep',i,'...');
    sampleset = dynex.sample(bqm, mainnet=True, num_reads=20000, annealing_time = 120000, logging=False)
    e = sampleset.first.energy/factor;
    if e < best_cut:
        best_cut = e;
        best_sol = sampleset.first;
        print('IMPROVED CUT FOUND:',best_cut);
        # save solution locally:
        with open('maxcut_best_'+str(round(abs(e)))+'.pkl','wb') as outp:
            pickle.dump(best_sol.sample, outp)
    all_sol.append(e);
    print('Sweep',i,'cut:',e,'best cut:',best_cut, 'all:',all_sol)

In [8]:
# load best solution:
with open('maxcut_best_9506.pkl','rb') as inp:
    x = pickle.load(inp)

In [9]:
# Validate cut result:
lut = x

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut['X['+str(node)+']']]
S1 = [node for node in G.nodes if lut['X['+str(node)+']']]
cut_edges = [(u, v) for u, v in G.edges if lut['X['+str(u)+']'] != lut['X['+str(v)+']']]
uncut_edges = [(u, v) for u, v in G.edges if lut['X['+str(u)+']'] == lut['X['+str(v)+']']]

print('Maxcut result:',len(cut_edges))

Maxcut result: 9506
